# Exploratory Data Analysis (EDA) and Business Insights

## Importing the required libraries for EDA

In [67]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

## Loading the data into the data frame

In [68]:
# File paths
customers_file = '/content/Customers.csv'
products_file = '/content/Products.csv'
transactions_file = '/content/Transactions.csv'

# Loading datasets
customers_df = pd.read_csv(customers_file)
products_df = pd.read_csv(products_file)
transactions_df = pd.read_csv(transactions_file)

print("Customers Dataset:")
display(customers_df.head())

print("\nProducts Dataset:")
display(products_df.head())

print("\nTransactions Dataset:")
display(transactions_df.head())

Customers Dataset:


,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15



Products Dataset:


,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31



Transactions Dataset:


,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [69]:
# Convert date columns to datetime format
customers_df["SignupDate"] = pd.to_datetime(customers_df["SignupDate"])
transactions_df["TransactionDate"] = pd.to_datetime(transactions_df["TransactionDate"])

# Verify TotalValue consistency i.e. TotalValue should be equal to Price * Quantity
transactions_df["CalculatedTotal"] = transactions_df["Price"] * transactions_df["Quantity"]
total_value_mismatch = transactions_df[transactions_df["TotalValue"] != transactions_df["CalculatedTotal"]]

# Check for duplicate records
customers_duplicates = customers_df.duplicated().sum()
products_duplicates = products_df.duplicated().sum()
transactions_duplicates = transactions_df.duplicated().sum()

# Check for duplicate IDs
customer_id_duplicates = customers_df["CustomerID"].duplicated().sum()
product_id_duplicates = products_df["ProductID"].duplicated().sum()
transaction_id_duplicates = transactions_df["TransactionID"].duplicated().sum()

{
    "Total Value Mismatch Count": total_value_mismatch.shape[0],
    "Duplicate Rows": {
        "Customers": customers_duplicates,
        "Products": products_duplicates,
        "Transactions": transactions_duplicates,
    },
    "Duplicate IDs": {
        "Customers": customer_id_duplicates,
        "Products": product_id_duplicates,
        "Transactions": transaction_id_duplicates,
    },
}


{'Total Value Mismatch Count': 88,
 'Duplicate Rows': {'Customers': 0, 'Products': 0, 'Transactions': 0},
 'Duplicate IDs': {'Customers': 0, 'Products': 0, 'Transactions': 0}}

In [70]:
mismatched_transactions = transactions_df[transactions_df["TotalValue"] != transactions_df["CalculatedTotal"]]
print(mismatched_transactions)

    TransactionID CustomerID ProductID     TransactionDate  Quantity  \
17         T00270      C0101      P034 2024-11-07 02:48:08         3   
29         T00218      C0148      P057 2024-01-17 19:40:55         3   
30         T00417      C0035      P057 2024-04-20 22:54:54         3   
31         T00492      C0120      P057 2024-08-08 05:40:02         3   
35         T00703      C0092      P057 2024-02-04 00:31:54         3   
..            ...        ...       ...                 ...       ...   
924        T00823      C0095      P079 2024-09-30 10:45:06         3   
935        T00660      C0057      P008 2024-09-23 16:46:01         3   
946        T00646      C0036      P091 2024-01-23 12:53:51         3   
947        T00793      C0054      P091 2024-10-07 17:48:28         3   
948        T00798      C0015      P091 2024-09-21 01:39:03         3   

     TotalValue   Price  CalculatedTotal  
17       651.15  217.05           651.15  
29       719.10  239.70           719.10  
30    

In [71]:
transactions_df.loc[transactions_df["TotalValue"] != transactions_df["CalculatedTotal"], "TotalValue"] = transactions_df["CalculatedTotal"]


In [72]:
transactions_customers = pd.merge(transactions_df, customers_df, on="CustomerID", how="left")
full_data = pd.merge(transactions_customers, products_df, on="ProductID", how="left")
full_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,CalculatedTotal,CustomerName,Region,SignupDate,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,601.36,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,902.04,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [73]:
full_data = full_data.drop(columns=['Price_y']).rename(columns={"Price_x": "ProductPrice"})

In [74]:
full_data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,ProductPrice,CalculatedTotal,CustomerName,Region,SignupDate,ProductName,Category
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,300.68,Andrea Jenkins,Europe,2022-12-03,ComfortLiving Bluetooth Speaker,Electronics
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,300.68,Brittany Harvey,Asia,2024-09-04,ComfortLiving Bluetooth Speaker,Electronics
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,300.68,Kathryn Stevens,Europe,2024-04-04,ComfortLiving Bluetooth Speaker,Electronics
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,601.36,Travis Campbell,South America,2024-04-11,ComfortLiving Bluetooth Speaker,Electronics
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,902.04,Timothy Perez,Europe,2022-03-15,ComfortLiving Bluetooth Speaker,Electronics


In [75]:
missing_values = full_data.isnull().sum()
missing_values

,0
TransactionID,0
CustomerID,0
ProductID,0
TransactionDate,0
Quantity,0
TotalValue,0
ProductPrice,0
CalculatedTotal,0
CustomerName,0
Region,0


In [76]:
full_data.shape

(1000, 13)

In [77]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

# Encoding categorical variables
encoder = LabelEncoder()
full_data["RegionEncoded"] = encoder.fit_transform(full_data["Region"])
full_data["CategoryEncoded"] = encoder.fit_transform(full_data["Category"])

# Add new features
full_data['TransactionDate'] = pd.to_datetime(full_data['TransactionDate'])
latest_date = full_data['TransactionDate'].max()

customer_profiles = full_data.groupby("CustomerID").agg({
    "RegionEncoded": "first",  # Categorical variable
    "ProductPrice": "mean",  # Average product price
    "Quantity": "sum",  # Total quantity purchased
    "CategoryEncoded": lambda x: x.mode()[0],  # Most purchased product category
    "TransactionID": "count",  # Purchase frequency
    "TransactionDate": lambda x: (latest_date - x.max()).days,  # Recency in days
    "Category": "nunique"  # Diversity of product categories
}).reset_index()

customer_profiles.rename(columns={"TransactionID": "Frequency", "Category": "Diversity", "TransactionDate": "Recency"}, inplace=True)

# Normalize features
scaler = MinMaxScaler()
feature_columns = ["RegionEncoded", "ProductPrice", "Quantity", "CategoryEncoded", "Frequency", "Recency", "Diversity"]
customer_profiles[feature_columns] = scaler.fit_transform(customer_profiles[feature_columns])

# Compute weighted similarity
weights = {
    "RegionEncoded": 0.1,
    "ProductPrice": 0.2,
    "Quantity": 0.2,
    "CategoryEncoded": 0.2,
    "Frequency": 0.15,
    "Recency": 0.1,
    "Diversity": 0.05
}
weighted_features = customer_profiles[feature_columns].copy()
for col, weight in weights.items():
    weighted_features[col] *= weight

similarity_matrix = cosine_similarity(weighted_features)

# Functions for lookalike recommendations
def get_top_lookalikes(customer_idx, similarity_matrix, customer_profiles, top_n=3):
    scores = list(enumerate(similarity_matrix[customer_idx]))
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    top_scores = scores[1:top_n+1]  # Exclude self
    return [(customer_profiles.iloc[idx]["CustomerID"], score) for idx, score in top_scores]

def recommend_similar_customers(customer_id, similarity_matrix, customer_profiles, top_n=3):
    if customer_id not in customer_profiles["CustomerID"].values:
        return f"CustomerID {customer_id} not found in the data."
    customer_idx = customer_profiles[customer_profiles["CustomerID"] == customer_id].index[0]
    lookalikes = get_top_lookalikes(customer_idx, similarity_matrix, customer_profiles, top_n)
    return {"CustomerID": customer_id, "TopLookalikes": lookalikes}

# # Get user input for the number of customers to process or default to 20
# num_customers = int(input("Enter the number of customers to process : ") or 20)

# # Ensure the dataset has enough customers
# if num_customers > len(customer_profiles):
#     print(f"The dataset contains only {len(customer_profiles)} customers. Using all available customers instead.")
#     num_customers = len(customer_profiles)


customer_id = input("Enter the CustomerID for lookalike recommendations: ")

# Check and recommend lookalikes for the given customer ID
if customer_id in customer_profiles["CustomerID"].values:
    result = recommend_similar_customers(customer_id, similarity_matrix, customer_profiles)
    print(f"Lookalike recommendations for CustomerID {customer_id}:")
    for lookalike, score in result["TopLookalikes"]:
        print(f"- CustomerID: {lookalike}, Similarity Score: {score:.4f}")
else:
    print(f"CustomerID {customer_id} not found in the data.")


# Generate lookalikes for the specified number of customers
lookalikes = {}
for customer_id in customer_profiles["CustomerID"].iloc[:num_customers]:
    result = recommend_similar_customers(customer_id, similarity_matrix, customer_profiles)
    lookalikes[customer_id] = result["TopLookalikes"]

# Save results to Lookalike.csv
lookalikes_df = pd.DataFrame({
    "CustomerID": list(lookalikes.keys()),
    "Lookalikes": [str(lookalikes[cust_id]) for cust_id in lookalikes]
})

# output_file = '/content/Aastha_Pal_Lookalike.csv'
# lookalikes_df.to_csv(output_file, index=False)

# print(f"Lookalike recommendations saved to {output_file}")


Enter the CustomerID for lookalike recommendations: C0001
Lookalike recommendations for CustomerID C0001:
- CustomerID: C0181, Similarity Score: 0.9901
- CustomerID: C0048, Similarity Score: 0.9897
- CustomerID: C0148, Similarity Score: 0.9883
